# Gleam
This notebook aims to design, train, validate, and use a convolutional neural network that is able to predict the distribution of population of a geographical area from nighttime satellite imagery. The product is a high resolution approximation map of .

# Datasets

[Recommended source nighttime pictures (NOAA)](https://ngdc.noaa.gov/eog/viirs/download_dnb_composites.html)

[Recommended source population dataset (SEDAC)](http://sedac.ciesin.columbia.edu/data/set/gpw-v4-population-count-adjusted-to-2015-unwpp-country-totals-rev10)

This data needs to be processed with a GIS tool to meet a few requirements before training :
- Rasters of the same year need to be merged in one GeoTIFF file : lights on the first band, population on the second band.
- Since both sets don't have the same resolution, upscaling the population set might have increased population density. This needs to be adjusted before training (use `rescale_pop.py` in the same folder as this notebook, don't forget to change some variables). In the case of the recommended datasets, every population pixel needs to be devided by 4 after merging with gdal_merge or QGIS.
- Use a vector file to clip the raster and isolate the region of interest ([recommended vector file to clip by countries (Natural Earth)](https://www.naturalearthdata.com/downloads/10m-cultural-vectors/10m-admin-0-countries/)). Areas with a population of zero (out of borders or in the ocean) will be ignored during preprocessing.

# Trained models
A few already trained models can be found in `./models`. These were trained on the recommended datasets using the scripts below for the year 2015, so the input dataset for a prediction must have 1 pixel per 0.25 square kilometer. The name of the model indicates the contries that were used to train it (`safrica_namibia.h5` is South Africa and Namibia).

# Imports

In [1]:
import rasterio
import numpy as np
import keras.layers.core as core
import keras.layers.convolutional as conv
import keras.models as models
import keras.callbacks
from sklearn.model_selection import KFold
from keras import optimizers
import time

C:\Users\Bertral\venv\cnn_pop\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Preprocessing
The preprocess function creates a window sliding over the input raster.

In [2]:
def preprocess(filepath, input_tile_size, offset):
    """
    :param filepath: GeoTIFF raster file with 2 bands
    :param input_tile_size: width of the square sliding window, and of the output tiles
    :param offset: distance in pixels the window will move between each tile
    :return: (X, Y) where X is a suitable array of inputs for the neural network
            and Y is the expected output for each of these inputs
    This function filters out the tiles that have zero population on them.
    """
    
    raster = rasterio.open(filepath)

    matrix_x = raster.read(1)
    matrix_y = raster.read(2)

    X = []
    Y = []
    col = 0
    while col + input_tile_size < matrix_x.shape[1]:
        row = 0
        while row + input_tile_size < matrix_x.shape[0]:
            pop = np.sum(matrix_y[row: row + input_tile_size, col: col + input_tile_size])
            # only use tiles that have people living on it
            if pop > 0:
                X.append(matrix_x[row: row + input_tile_size, col: col + input_tile_size])
                Y.append(pop)

            row += offset
        col += offset

    raster.close()
    matrix_x, matrix_y = None, None  # free some memory
    X, Y = np.array(X), np.array(Y)
    X = np.expand_dims(X, axis=3)  # add the color channel as a new dimension
    print('input shape (observations, obs_width, obs_height, channels) : ' + str(X.shape))
    return X, Y


# The neural network
This part of the script defines how the neural network will be initialized and creates callbacks.

In [3]:
def init_cnn(input_shape):
    # kernel size for each convolution layer
    kernel_size = (3, 3)
    
    cnn = models.Sequential()

    cnn.add(conv.Convolution2D(filters=64, kernel_size=kernel_size, activation="relu", padding='same',
                               input_shape=input_shape))
    cnn.add(conv.AveragePooling2D(strides=(2, 2)))
    
    cnn.add(conv.Convolution2D(filters=128, kernel_size=kernel_size, activation="relu", padding='same'))
    cnn.add(conv.AveragePooling2D(strides=(2, 2)))

    cnn.add(conv.Convolution2D(filters=256, kernel_size=kernel_size, activation="relu", padding='same'))
    cnn.add(conv.AveragePooling2D(strides=(2, 2)))
    
    cnn.add(core.Flatten())
    cnn.add(core.Dropout(0.5))
    cnn.add(core.Dense(128))
    cnn.add(core.Dense(1))

    cnn.compile(loss="mean_squared_error", optimizer=optimizers.Adam(lr=0.02, decay=0.0), metrics=["mse", "mae"])
    return cnn

# CALLBACKS
# reduce learning rate when we stopped learning anything
rlrp = keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.5, patience=20, verbose=1, mode='auto', min_lr=0.0000001)

# stop learning early if we stopped leaning anything for a longer time
early_stopping = keras.callbacks.EarlyStopping(monitor='loss', min_delta=0.0001, patience=200, verbose=1, mode='auto')

# K-Fold validation
This script trains and validates the model using 4-fold cross-validation. This script was used to find the best performing neural network topology and measure the amount of error we can expect from a trained model. The resulting model is saved to the models/ subfolder.

In [5]:
## PARAMETERS ##
nb_epoch = 2000  # maximum number of epochs
model_birthday = time.strftime("%Y-%m-%d_%H-%M-%S", time.gmtime())  # used to identify generated files (logs and models)
raster_path = '../data/lightpop_merged/adj_2015_usa.tif'  # raster containing the training dataset
verbose = 2  # 0: no progress message, 1: progress bar for each epoch, 2: one message for each epoch
################

# CALLBACKS
# logs for tensorboard
tensorboard = keras.callbacks.TensorBoard(log_dir="logs/" + model_birthday)

print('preprocessing')
X, Y = preprocess(raster_path, 32, 32)

print('configuring cnn')

# input dimensions
img_count, img_rows, img_cols, img_channel_count = X.shape

# k-fold split
kfold = KFold(n_splits=4, shuffle=True, random_state=None)

# initialize statistics aggregate
kfold_mse = []
kfold_mae = []
kfold_sae = []

print('logs will be saved to logs/' + model_birthday)

for train, test in kfold.split(X, Y):
    
    cnn = init_cnn((img_rows, img_cols, img_channel_count))

    print('training ...')

    cnn.fit(X[train], Y[train], batch_size=1024, epochs=nb_epoch, verbose=verbose, callbacks=[tensorboard, rlrp, early_stopping],
            sample_weight=None)

    cnn.save('models/' + model_birthday + '.h5')

    print('model saved to models/' + model_birthday + '.h5')
    
    # evaluate and print stats
    evaluation = cnn.evaluate(X[test], Y[test], verbose=2, batch_size=1024)
    evaluation = dict(zip(cnn.metrics_names, evaluation))
    kfold_mse.append(evaluation['mean_squared_error'])
    kfold_mae.append(evaluation['mean_absolute_error'])
    kfold_sae.append(evaluation['mean_absolute_error'] * len(Y[test]))
    
    print('K-fold validation results :')
    print('Mean squared error : %.2f (std %.2f)' % (np.mean(kfold_mse), np.std(kfold_mse)))
    print('Mean absolute error : %.2f (std %.2f)' % (np.mean(kfold_mae), np.std(kfold_mae)))
    print('Sum of absolute errors : %.2f (std %.2f)' % (np.mean(kfold_sae), np.std(kfold_sae)))

print('done !')


preprocessing
input shape (observations, obs_width, obs_height, channels) : (50204, 32, 32, 1)
configuring cnn
logs will be saved to logs/2018-07-15_06-46-43
training ...
Epoch 1/2000
 - 3s - loss: 1206809642.0922 - mean_squared_error: 1206809642.0922 - mean_absolute_error: 6723.8641
Epoch 2/2000
 - 3s - loss: 1125980749.8800 - mean_squared_error: 1125980749.8800 - mean_absolute_error: 6664.4744
Epoch 3/2000
 - 3s - loss: 725990322.3486 - mean_squared_error: 725990322.3486 - mean_absolute_error: 7703.2521
Epoch 4/2000
 - 3s - loss: 426352526.6559 - mean_squared_error: 426352526.6559 - mean_absolute_error: 4775.8316
Epoch 5/2000
 - 3s - loss: 400593118.1688 - mean_squared_error: 400593118.1688 - mean_absolute_error: 4344.4397
Epoch 6/2000
 - 3s - loss: 411070058.6004 - mean_squared_error: 411070058.6004 - mean_absolute_error: 3964.2102
Epoch 7/2000
 - 3s - loss: 401508613.8428 - mean_squared_error: 401508613.8428 - mean_absolute_error: 3680.7651
Epoch 8/2000
 - 3s - loss: 384345509.0151

 - 3s - loss: 235744163.0795 - mean_squared_error: 235744163.0795 - mean_absolute_error: 3321.7134
Epoch 73/2000
 - 3s - loss: 231176794.9407 - mean_squared_error: 231176794.9407 - mean_absolute_error: 3333.6170
Epoch 74/2000
 - 3s - loss: 216690948.2476 - mean_squared_error: 216690948.2476 - mean_absolute_error: 3293.2587
Epoch 75/2000
 - 3s - loss: 181303754.9407 - mean_squared_error: 181303754.9407 - mean_absolute_error: 3078.3867
Epoch 76/2000
 - 3s - loss: 207732352.4997 - mean_squared_error: 207732352.4997 - mean_absolute_error: 3282.3207
Epoch 77/2000
 - 3s - loss: 224493158.0328 - mean_squared_error: 224493158.0328 - mean_absolute_error: 3408.7559
Epoch 78/2000
 - 3s - loss: 216737951.7548 - mean_squared_error: 216737951.7548 - mean_absolute_error: 3043.5294
Epoch 79/2000
 - 3s - loss: 212486949.2398 - mean_squared_error: 212486949.2398 - mean_absolute_error: 3345.7602
Epoch 80/2000
 - 3s - loss: 197209413.3227 - mean_squared_error: 197209413.3227 - mean_absolute_error: 2927.30

Epoch 144/2000
 - 3s - loss: 119011104.4436 - mean_squared_error: 119011104.4436 - mean_absolute_error: 2696.9458
Epoch 145/2000
 - 3s - loss: 117517095.6086 - mean_squared_error: 117517095.6086 - mean_absolute_error: 2636.3214
Epoch 146/2000
 - 3s - loss: 116011473.8096 - mean_squared_error: 116011473.8096 - mean_absolute_error: 2666.3177
Epoch 147/2000
 - 3s - loss: 117070255.7455 - mean_squared_error: 117070255.7455 - mean_absolute_error: 2566.0245
Epoch 148/2000
 - 3s - loss: 100814183.9832 - mean_squared_error: 100814183.9832 - mean_absolute_error: 2534.4161
Epoch 149/2000
 - 3s - loss: 107712172.7374 - mean_squared_error: 107712172.7374 - mean_absolute_error: 2540.2862
Epoch 150/2000
 - 3s - loss: 111302276.2478 - mean_squared_error: 111302276.2478 - mean_absolute_error: 2576.8549
Epoch 151/2000
 - 3s - loss: 104231741.0993 - mean_squared_error: 104231741.0993 - mean_absolute_error: 2517.0895
Epoch 152/2000
 - 3s - loss: 97227908.5076 - mean_squared_error: 97227908.5076 - mean_ab

Epoch 217/2000
 - 3s - loss: 106603943.2665 - mean_squared_error: 106603943.2665 - mean_absolute_error: 2553.9123
Epoch 218/2000
 - 3s - loss: 95513467.8431 - mean_squared_error: 95513467.8431 - mean_absolute_error: 2489.4108
Epoch 219/2000
 - 3s - loss: 134279906.6554 - mean_squared_error: 134279906.6554 - mean_absolute_error: 2697.7054
Epoch 220/2000
 - 3s - loss: 136081382.0381 - mean_squared_error: 136081382.0381 - mean_absolute_error: 2707.5963
Epoch 221/2000
 - 3s - loss: 92168717.4034 - mean_squared_error: 92168717.4034 - mean_absolute_error: 2484.3627
Epoch 222/2000
 - 3s - loss: 85207036.5034 - mean_squared_error: 85207036.5034 - mean_absolute_error: 2372.3685
Epoch 223/2000
 - 3s - loss: 88235414.8131 - mean_squared_error: 88235414.8131 - mean_absolute_error: 2447.6670
Epoch 224/2000
 - 3s - loss: 87890219.4224 - mean_squared_error: 87890219.4224 - mean_absolute_error: 2349.2089
Epoch 225/2000
 - 3s - loss: 77216552.0869 - mean_squared_error: 77216552.0869 - mean_absolute_err

Epoch 290/2000
 - 3s - loss: 55079521.8092 - mean_squared_error: 55079521.8092 - mean_absolute_error: 2173.5259
Epoch 291/2000
 - 3s - loss: 54830549.2973 - mean_squared_error: 54830549.2973 - mean_absolute_error: 2149.6953
Epoch 292/2000
 - 3s - loss: 55728470.2315 - mean_squared_error: 55728470.2315 - mean_absolute_error: 2173.4646
Epoch 293/2000
 - 3s - loss: 54620092.6380 - mean_squared_error: 54620092.6380 - mean_absolute_error: 2127.2494
Epoch 294/2000
 - 3s - loss: 59220681.5746 - mean_squared_error: 59220681.5746 - mean_absolute_error: 2182.9998
Epoch 295/2000
 - 3s - loss: 60407280.1711 - mean_squared_error: 60407280.1711 - mean_absolute_error: 2178.5031
Epoch 296/2000
 - 3s - loss: 54709867.8621 - mean_squared_error: 54709867.8621 - mean_absolute_error: 2156.7461
Epoch 297/2000
 - 3s - loss: 60324443.9223 - mean_squared_error: 60324443.9223 - mean_absolute_error: 2148.1949
Epoch 298/2000
 - 3s - loss: 54663330.0446 - mean_squared_error: 54663330.0446 - mean_absolute_error: 21

...

Epoch 425/2000
 - 3s - loss: 19899648.3206 - mean_squared_error: 19899648.3206 - mean_absolute_error: 1539.1422
Epoch 426/2000
 - 3s - loss: 19494164.9159 - mean_squared_error: 19494164.9159 - mean_absolute_error: 1533.4656
Epoch 427/2000
 - 3s - loss: 20169539.5084 - mean_squared_error: 20169539.5084 - mean_absolute_error: 1528.0922
Epoch 428/2000
 - 3s - loss: 20341761.2546 - mean_squared_error: 20341761.2546 - mean_absolute_error: 1536.7939
Epoch 429/2000
 - 3s - loss: 20333678.0576 - mean_squared_error: 20333678.0576 - mean_absolute_error: 1539.5847
Epoch 430/2000
 - 3s - loss: 18566826.0647 - mean_squared_error: 18566826.0647 - mean_absolute_error: 1517.7690
Epoch 431/2000
 - 3s - loss: 19432341.4154 - mean_squared_error: 19432341.4154 - mean_absolute_error: 1535.7737
Epoch 432/2000
 - 3s - loss: 19344069.2193 - mean_squared_error: 19344069.2193 - mean_absolute_error: 1530.0667
Epoch 433/2000
 - 3s - loss: 19541616.9410 - mean_squared_error: 19541616.9410 - mean_absolute_error: 15

 - 3s - loss: 19393789.1595 - mean_squared_error: 19393789.1595 - mean_absolute_error: 1528.8954
Epoch 497/2000
 - 3s - loss: 18654202.5493 - mean_squared_error: 18654202.5493 - mean_absolute_error: 1519.4084
Epoch 498/2000
 - 3s - loss: 19592014.8884 - mean_squared_error: 19592014.8884 - mean_absolute_error: 1519.7660
Epoch 499/2000
 - 3s - loss: 19273997.1922 - mean_squared_error: 19273997.1922 - mean_absolute_error: 1539.6185
Epoch 500/2000
 - 3s - loss: 19840948.1183 - mean_squared_error: 19840948.1183 - mean_absolute_error: 1517.3436
Epoch 501/2000
 - 3s - loss: 18524328.2374 - mean_squared_error: 18524328.2374 - mean_absolute_error: 1517.8511
Epoch 502/2000
 - 3s - loss: 19418626.0677 - mean_squared_error: 19418626.0677 - mean_absolute_error: 1544.8975
Epoch 503/2000
 - 3s - loss: 20198700.1782 - mean_squared_error: 20198700.1782 - mean_absolute_error: 1539.4369
Epoch 504/2000
 - 3s - loss: 18899064.0828 - mean_squared_error: 18899064.0828 - mean_absolute_error: 1520.5965
Epoch 5

 - 3s - loss: 19004253.8646 - mean_squared_error: 19004253.8646 - mean_absolute_error: 1518.6079
Epoch 568/2000
 - 3s - loss: 19268202.6882 - mean_squared_error: 19268202.6882 - mean_absolute_error: 1509.2941
Epoch 569/2000
 - 3s - loss: 19074717.8535 - mean_squared_error: 19074717.8535 - mean_absolute_error: 1526.9530
Epoch 570/2000
 - 3s - loss: 19630032.5129 - mean_squared_error: 19630032.5129 - mean_absolute_error: 1530.7318
Epoch 571/2000
 - 3s - loss: 21795433.7568 - mean_squared_error: 21795433.7568 - mean_absolute_error: 1543.0486
Epoch 572/2000
 - 3s - loss: 17974055.0782 - mean_squared_error: 17974055.0782 - mean_absolute_error: 1514.4279
Epoch 573/2000
 - 3s - loss: 19958448.1239 - mean_squared_error: 19958448.1239 - mean_absolute_error: 1542.6628
Epoch 574/2000
 - 3s - loss: 18817024.7053 - mean_squared_error: 18817024.7053 - mean_absolute_error: 1538.3657
Epoch 575/2000
 - 3s - loss: 18630877.1081 - mean_squared_error: 18630877.1081 - mean_absolute_error: 1524.2319
Epoch 5

 - 3s - loss: 20008920.5401 - mean_squared_error: 20008920.5401 - mean_absolute_error: 1543.1853
Epoch 639/2000
 - 3s - loss: 17364700.9834 - mean_squared_error: 17364700.9834 - mean_absolute_error: 1515.8082
Epoch 640/2000
 - 3s - loss: 18667716.5220 - mean_squared_error: 18667716.5220 - mean_absolute_error: 1520.0319
Epoch 641/2000
 - 3s - loss: 18748753.5423 - mean_squared_error: 18748753.5423 - mean_absolute_error: 1506.4115
Epoch 642/2000
 - 3s - loss: 18446865.3302 - mean_squared_error: 18446865.3302 - mean_absolute_error: 1511.7594
Epoch 643/2000
 - 3s - loss: 18511191.8005 - mean_squared_error: 18511191.8005 - mean_absolute_error: 1505.0781
Epoch 644/2000
 - 3s - loss: 18829877.6032 - mean_squared_error: 18829877.6032 - mean_absolute_error: 1521.3532
Epoch 645/2000
 - 3s - loss: 18911383.3901 - mean_squared_error: 18911383.3901 - mean_absolute_error: 1520.3536

Epoch 00645: ReduceLROnPlateau reducing learning rate to 3.0517577442878974e-07.
Epoch 646/2000
 - 3s - loss: 18463323

KeyboardInterrupt: 

# Training without validation
Trains on a whole raster once, without validating. The goal is to build a model that makes the best possible prediction, so it has to train on every data available including the validation set. The resulting model is saved to the models/ subfolder.

In [5]:
## PARAMETERS ##
nb_epoch = 5000  # maximum number of epochs
model_birthday = time.strftime("%Y-%m-%d_%H-%M-%S", time.gmtime())  # used to identify generated files (logs and models)
raster_path = '../data/lightpop_merged/adj_2015_usa.tif'  # raster containing the training dataset
verbose = 2  # 0: no progress message, 1: progress bar for each epoch, 2: one message for each epoch
################

# CALLBACKS
# logs for tensorboard
tensorboard = keras.callbacks.TensorBoard(log_dir="logs/" + model_birthday)

# checkpoints in case of crash or interruption
checkpoint = keras.callbacks.ModelCheckpoint('models/' + model_birthday + '.h5', save_weights_only=False)

print('preprocessing')

X, Y = preprocess(raster_path, 32, 8)

print('configuring cnn')

# input dimensions
img_count, img_rows, img_cols, img_channel_count = X.shape

print('logs will be saved to logs/' + model_birthday)

cnn = init_cnn((img_rows, img_cols, img_channel_count))

print('training ...')

cnn.fit(X, Y, batch_size=1024, epochs=nb_epoch, verbose=verbose,
        callbacks=[tensorboard, checkpoint, rlrp, early_stopping], sample_weight=None)

cnn.save('models/' + model_birthday + '.h5')

print('model saved to models/' + model_birthday + '.h5')

print('done !')

preprocessing
input shape (observations, obs_width, obs_height, channels) : (803982, 32, 32, 1)
configuring cnn
logs will be saved to logs/2018-07-15_21-36-36
training ...
Epoch 1/5000
 - 74s - loss: 404535320.1127 - mean_squared_error: 404535320.1127 - mean_absolute_error: 3812.5784
Epoch 2/5000
 - 69s - loss: 318896448.0157 - mean_squared_error: 318896448.0157 - mean_absolute_error: 3782.9507
Epoch 3/5000
 - 69s - loss: 263632261.9791 - mean_squared_error: 263632261.9791 - mean_absolute_error: 3546.7765
Epoch 4/5000
 - 69s - loss: 291624038.6583 - mean_squared_error: 291624038.6583 - mean_absolute_error: 3583.6554
Epoch 5/5000
 - 69s - loss: 290324285.0619 - mean_squared_error: 290324285.0619 - mean_absolute_error: 3796.6624
Epoch 6/5000
 - 69s - loss: 248989262.0820 - mean_squared_error: 248989262.0820 - mean_absolute_error: 3592.4387
Epoch 7/5000
 - 69s - loss: 252921102.6044 - mean_squared_error: 252921102.6044 - mean_absolute_error: 3612.9277
Epoch 8/5000
 - 69s - loss: 264296026

Epoch 72/5000
 - 73s - loss: 81250646.7854 - mean_squared_error: 81250646.7854 - mean_absolute_error: 2371.7518
Epoch 73/5000
 - 73s - loss: 72421546.1494 - mean_squared_error: 72421546.1494 - mean_absolute_error: 2295.1849
Epoch 74/5000
 - 73s - loss: 78486156.4020 - mean_squared_error: 78486156.4020 - mean_absolute_error: 2325.3945
Epoch 75/5000
 - 73s - loss: 90977447.2681 - mean_squared_error: 90977447.2681 - mean_absolute_error: 2464.8808
Epoch 76/5000
 - 73s - loss: 68516032.8531 - mean_squared_error: 68516032.8531 - mean_absolute_error: 2281.2922
Epoch 77/5000
 - 73s - loss: 63694347.5137 - mean_squared_error: 63694347.5137 - mean_absolute_error: 2242.2471
Epoch 78/5000
 - 73s - loss: 65320832.6728 - mean_squared_error: 65320832.6728 - mean_absolute_error: 2233.5865
Epoch 79/5000
 - 73s - loss: 69810292.4908 - mean_squared_error: 69810292.4908 - mean_absolute_error: 2263.3623
Epoch 80/5000
 - 73s - loss: 67952247.6273 - mean_squared_error: 67952247.6273 - mean_absolute_error: 22

 - 73s - loss: 40390921.2532 - mean_squared_error: 40390921.2532 - mean_absolute_error: 1969.7621
Epoch 145/5000
 - 73s - loss: 38148826.9160 - mean_squared_error: 38148826.9160 - mean_absolute_error: 1945.6616
Epoch 146/5000
 - 73s - loss: 38795523.8738 - mean_squared_error: 38795523.8738 - mean_absolute_error: 1940.1809
Epoch 147/5000
 - 73s - loss: 36912248.7788 - mean_squared_error: 36912248.7788 - mean_absolute_error: 1925.8787
Epoch 148/5000
 - 73s - loss: 41880173.2699 - mean_squared_error: 41880173.2699 - mean_absolute_error: 1977.0210
Epoch 149/5000
 - 73s - loss: 38899969.6227 - mean_squared_error: 38899969.6227 - mean_absolute_error: 1930.1542
Epoch 150/5000
 - 73s - loss: 38956533.5556 - mean_squared_error: 38956533.5556 - mean_absolute_error: 1941.3319
Epoch 151/5000
 - 73s - loss: 39580408.3115 - mean_squared_error: 39580408.3115 - mean_absolute_error: 1937.8539
Epoch 152/5000
 - 73s - loss: 37677138.7923 - mean_squared_error: 37677138.7923 - mean_absolute_error: 1929.780

Epoch 217/5000
 - 73s - loss: 32264939.9629 - mean_squared_error: 32264939.9629 - mean_absolute_error: 1807.2898
Epoch 218/5000
 - 73s - loss: 32682871.4944 - mean_squared_error: 32682871.4944 - mean_absolute_error: 1816.7120
Epoch 219/5000
 - 73s - loss: 35698225.4494 - mean_squared_error: 35698225.4494 - mean_absolute_error: 1860.2564
Epoch 220/5000
 - 73s - loss: 33363261.2835 - mean_squared_error: 33363261.2835 - mean_absolute_error: 1831.2835
Epoch 221/5000
 - 73s - loss: 33957261.8035 - mean_squared_error: 33957261.8035 - mean_absolute_error: 1829.5891
Epoch 222/5000
 - 73s - loss: 36049371.6877 - mean_squared_error: 36049371.6877 - mean_absolute_error: 1862.8895
Epoch 223/5000
 - 73s - loss: 34227082.5177 - mean_squared_error: 34227082.5177 - mean_absolute_error: 1831.9928
Epoch 224/5000
 - 72s - loss: 32153241.3976 - mean_squared_error: 32153241.3976 - mean_absolute_error: 1803.8849
Epoch 225/5000
 - 73s - loss: 46826829.0475 - mean_squared_error: 46826829.0475 - mean_absolute_

Epoch 289/5000
 - 73s - loss: 27523823.5338 - mean_squared_error: 27523823.5338 - mean_absolute_error: 1706.3509
Epoch 290/5000
 - 72s - loss: 26927846.0304 - mean_squared_error: 26927846.0304 - mean_absolute_error: 1691.6194
Epoch 291/5000
 - 72s - loss: 26472700.5121 - mean_squared_error: 26472700.5121 - mean_absolute_error: 1688.0732
Epoch 292/5000
 - 72s - loss: 27017789.1135 - mean_squared_error: 27017789.1135 - mean_absolute_error: 1691.5784
Epoch 293/5000
 - 73s - loss: 26463959.9210 - mean_squared_error: 26463959.9210 - mean_absolute_error: 1687.4014
Epoch 294/5000
 - 73s - loss: 25419584.4801 - mean_squared_error: 25419584.4801 - mean_absolute_error: 1677.8565
Epoch 295/5000
 - 72s - loss: 26095889.4743 - mean_squared_error: 26095889.4743 - mean_absolute_error: 1682.1520
Epoch 296/5000
 - 72s - loss: 27027610.1564 - mean_squared_error: 27027610.1564 - mean_absolute_error: 1691.9610
Epoch 297/5000
 - 72s - loss: 26788838.0339 - mean_squared_error: 26788838.0339 - mean_absolute_

...

 - 72s - loss: 21276534.1753 - mean_squared_error: 21276534.1753 - mean_absolute_error: 1555.1324
Epoch 648/5000
 - 72s - loss: 21434344.6206 - mean_squared_error: 21434344.6206 - mean_absolute_error: 1554.1743
Epoch 649/5000
 - 72s - loss: 21489295.0981 - mean_squared_error: 21489295.0981 - mean_absolute_error: 1555.6778
Epoch 650/5000
 - 72s - loss: 21656407.6203 - mean_squared_error: 21656407.6203 - mean_absolute_error: 1555.9467
Epoch 651/5000
 - 72s - loss: 21187425.6567 - mean_squared_error: 21187425.6567 - mean_absolute_error: 1551.6878
Epoch 652/5000
 - 72s - loss: 21474364.5681 - mean_squared_error: 21474364.5681 - mean_absolute_error: 1556.1899
Epoch 653/5000
 - 72s - loss: 21511344.5681 - mean_squared_error: 21511344.5681 - mean_absolute_error: 1558.8491
Epoch 654/5000
 - 72s - loss: 21381558.9563 - mean_squared_error: 21381558.9563 - mean_absolute_error: 1553.3743
Epoch 655/5000
 - 72s - loss: 21304666.5310 - mean_squared_error: 21304666.5310 - mean_absolute_error: 1552.791

Epoch 718/5000
 - 72s - loss: 21341545.1905 - mean_squared_error: 21341545.1905 - mean_absolute_error: 1555.3148

Epoch 00718: ReduceLROnPlateau reducing learning rate to 1.220703097715159e-06.
Epoch 719/5000
 - 72s - loss: 21347100.5276 - mean_squared_error: 21347100.5276 - mean_absolute_error: 1556.3966
Epoch 720/5000
 - 72s - loss: 21346694.8888 - mean_squared_error: 21346694.8888 - mean_absolute_error: 1554.2120
Epoch 721/5000
 - 72s - loss: 21625037.7824 - mean_squared_error: 21625037.7824 - mean_absolute_error: 1559.9779
Epoch 722/5000
 - 73s - loss: 21426251.7061 - mean_squared_error: 21426251.7061 - mean_absolute_error: 1555.0453
Epoch 723/5000
 - 73s - loss: 21446568.3463 - mean_squared_error: 21446568.3463 - mean_absolute_error: 1555.1277
Epoch 724/5000
 - 71s - loss: 21285425.2021 - mean_squared_error: 21285425.2021 - mean_absolute_error: 1551.9771
Epoch 725/5000
 - 71s - loss: 21681502.1457 - mean_squared_error: 21681502.1457 - mean_absolute_error: 1558.5866
Epoch 726/5000


Epoch 788/5000
 - 74s - loss: 21626691.8471 - mean_squared_error: 21626691.8471 - mean_absolute_error: 1558.8946
Epoch 789/5000
 - 75s - loss: 21071119.7520 - mean_squared_error: 21071119.7520 - mean_absolute_error: 1555.5213
Epoch 790/5000
 - 75s - loss: 21446714.6598 - mean_squared_error: 21446714.6598 - mean_absolute_error: 1555.0764
Epoch 791/5000
 - 75s - loss: 21674907.4936 - mean_squared_error: 21674907.4936 - mean_absolute_error: 1555.4888
Epoch 792/5000
 - 75s - loss: 21359330.0116 - mean_squared_error: 21359330.0116 - mean_absolute_error: 1556.5001
Epoch 793/5000
 - 72s - loss: 21395714.9276 - mean_squared_error: 21395714.9276 - mean_absolute_error: 1555.2904
Epoch 794/5000
 - 71s - loss: 21346392.1978 - mean_squared_error: 21346392.1978 - mean_absolute_error: 1556.1757
Epoch 795/5000
 - 74s - loss: 21473126.3306 - mean_squared_error: 21473126.3306 - mean_absolute_error: 1552.5177
Epoch 796/5000
 - 75s - loss: 21713232.0378 - mean_squared_error: 21713232.0378 - mean_absolute_

Epoch 859/5000
 - 72s - loss: 21126769.6202 - mean_squared_error: 21126769.6202 - mean_absolute_error: 1554.8655
Epoch 860/5000
 - 72s - loss: 21337818.9390 - mean_squared_error: 21337818.9390 - mean_absolute_error: 1551.1452
Epoch 861/5000
 - 71s - loss: 21281012.6656 - mean_squared_error: 21281012.6656 - mean_absolute_error: 1554.3676
Epoch 862/5000
 - 72s - loss: 21823543.5356 - mean_squared_error: 21823543.5356 - mean_absolute_error: 1558.6698
Epoch 863/5000
 - 71s - loss: 21728698.5050 - mean_squared_error: 21728698.5050 - mean_absolute_error: 1557.0590

Epoch 00863: ReduceLROnPlateau reducing learning rate to 1e-07.
Epoch 864/5000
 - 73s - loss: 21151944.6278 - mean_squared_error: 21151944.6278 - mean_absolute_error: 1553.0422
Epoch 865/5000
 - 72s - loss: 21335657.2741 - mean_squared_error: 21335657.2741 - mean_absolute_error: 1558.9139
Epoch 866/5000
 - 74s - loss: 21229994.8040 - mean_squared_error: 21229994.8040 - mean_absolute_error: 1552.8413
Epoch 867/5000
 - 72s - loss: 2

KeyboardInterrupt: 

# Prediction
This script generates a high resolution population raster from a satellite image and a trained model. It uses its own preprocessing algorithm that works without validation data.

Since the model gives only one output for each 32x32 tile, the distribution of the output value in the tile is managed by this script. It assumes a log relation between nightlights and population for each pixel.

The notebook "rastercomparator" can then be used to compare actual data with predicted data, as well as the predicted population counts over the years.

In [4]:
## PARAMETERS ##
model_path = "models/usa.h5"
prediction_dataset = '../data/lightpop_merged/adj_2015_portugal.tif'  # input nightlights
out = '2015_usa_to_2015_portugal.tif'  # output population file
input_tile_size = 32
################

print('loading model')

cnn = models.load_model(model_path)

print('opening raster')

raster = rasterio.open(prediction_dataset)
band = raster.read(1)
profile = raster.profile
profile.update(count=1)
width, height = raster.width, raster.height

# preprocess
matrix_x = raster.read(1)
tiles_x = []
y = 0
while y + input_tile_size < matrix_x.shape[1]:
    x = 0
    while x + input_tile_size < matrix_x.shape[0]:
        tiles_x.append(matrix_x[x: x + input_tile_size, y: y + input_tile_size])
        x += input_tile_size
    y += input_tile_size
testX = np.array(tiles_x)
raster.close()
matrix_x = None
tiles_x = None
testX = np.expand_dims(testX, axis=3)

print('generating raster')

predicted_tiles = cnn.predict(testX, verbose=0)

predicted_raster = np.zeros(shape=(raster.height, raster.width))
y = 0
pred_index = 0
while y + input_tile_size < width:
    x = 0
    while x + input_tile_size < height:
        in_tile = band[x: x + input_tile_size, y: y + input_tile_size]
        if np.max(in_tile) <= 0:
            # avoid divisions by 0
            predicted_raster[x: x + input_tile_size, y: y + input_tile_size] = 0
        else:
            # normalize visible light between 0 and 1 to avoid overflows (also gives better results)
            weights = in_tile / np.max(in_tile)
            # visible light is perceived logarithmically => counteract with exp
            weights = np.exp(weights) - 1
            # the sum of all weights must be 1
            weights = weights / np.sum(weights)
            predicted_raster[x: x + input_tile_size, y: y + input_tile_size] = predicted_tiles[pred_index] * weights

        pred_index += 1
        x += input_tile_size
    y += input_tile_size

predicted_raster = np.array(predicted_raster)

with rasterio.open('predictions/' + out, 'w', **profile) as dst:
    dst.write(predicted_raster.astype(rasterio.float32), 1)
print("prediction saved to predictions/" + out)

predicted_raster = None

print('prediction done !')


loading model
opening raster
generating raster
prediction saved to predictions/2015_usa_to_2015_portugal.tif
prediction done !
